# Comp 551-Assignment 2


## Aymen Rumi-260661663
   
### ***-Refer to report for further detailed exlanation/observation/answers to each inidivudal question and their subquestions: this file contains mainly code and viualizations with explanation for my work, this file is still very comprehensive of the assignment-***



In [1]:
#all modules needed for assignment
import numpy as np
import csv
import pandas as pd
import matplotlib.pyplot as plt
import math
import random
import warnings
from scipy.spatial.distance import euclidean


# Better visualization for plots
plt.rcParams['figure.figsize'] = (20.0, 20.0)
plt.rcParams.update({'font.size': 22})

# Question 1: Generate Data

In [2]:
# function to import data given text file, adjusts to array dimensions and removes nan values
def getData(textFile):
    file=np.genfromtxt(textFile, delimiter=',')
    if(len(np.shape(file))==1):
        file = [x for x in file if str(x) != 'nan'] 
    else:
        cutoff=np.shape(file)[1]-1
        file = file[0:cutoff,0:cutoff]  
    return file


# function to generate multivariate data distribution with label, given mean vector, covariance matrix, and amount of data points
def generateMultiVariableData(sign,mean,covariance,amount):
    if(sign==-1):
        label='Negative'
        
    if(sign==1):
        label='Positive'
        
    data=[]
    for i in range(amount+1):
        data.append((label,np.random.multivariate_normal(mean, covariance)))
    
    return data

# function to split data classes into specified amount for train,validate, and test, and returns them
def splitData(test,validation,train,class1,class2):
    np.random.shuffle(class1)
    np.random.shuffle(class2)

    test_set = [i for i in class1[0:int((len(class1)*test))]] + [j for j in class2[0:int((len(class2)*test))]]
    np.random.shuffle(test_set)
    pd.DataFrame(test_set).to_csv('DS1_test') 
    validation_set = [x for x in class1[int((len(class1)*test)):int(((len(class1)*test)+(len(class1)*validation)))]] + [y for y in class2[int((len(class2)*test)):int(((len(class2)*test)+(len(class2)*validation)))]]
    np.random.shuffle(validation_set)
    pd.DataFrame(validation_set).to_csv('DS1_valid')  
    class1 = class1[int(((len(class1)*test)+(len(class1)*validation))):len(class1)]    
    class2 = class2[int(((len(class2)*test)+(len(class2)*validation))):len(class2)]   
    train_set = class1 + class2
    np.random.shuffle(train_set)
    
    pd.DataFrame(train_set).to_csv('DS1_train')
    
    return[test_set,validation_set,train_set]
    

# initializing mean vectors for positive and negative classification, and covariance matrix
meanVector_negative = getData('DS1_m_1.txt')
meanVector_positive=getData('DS1_m_0.txt')
covariance_matrix = getData('DS1_Cov.txt')


# initializing seperation class data
negative_class = generateMultiVariableData(-1,meanVector_negative,covariance_matrix,2000)
positive_class = generateMultiVariableData(1,meanVector_positive,covariance_matrix,2000)


#initializing all dataset
DS1 = positive_class + negative_class
np.random.shuffle(DS1)
pd.DataFrame(DS1).to_csv('DS1')


#intitializing test, validation, and trainingset
test_set=splitData(.2,.2,.6,negative_class,positive_class)[0]
validation_set=splitData(.2,.2,.6,negative_class,positive_class)[1]
train_set=splitData(.2,.2,.6,negative_class,positive_class)[2]


# Question 2: Gaussian Discriminant Analysis

In [3]:
# method to find phi given both classes
def Phi(positive_class,negative_class):
    return len(positive_class)/len(positive_class+negative_class)

#find mean matrix given class data
def meanMatrix(class_data):
    mean=[]
    for i in range(len(class_data[0][1])):
        value=0
        for data_point in class_data:
            value =value+ data_point[1][i]
        mean.append(value/len(class_data))
    return np.matrix(mean).transpose()
  
#find s value to plug into covariance function
def sValue(class_data):
    s = 0.0
    mean_matrix=meanMatrix(class_data)   
    for point in class_data:
        matrix = np.matrix(point[1]).transpose()
        matrix2 = (matrix - mean_matrix) * (matrix - mean_matrix).transpose()
        s= s+matrix2
    s = s/len(class_data)
    return s

#finding covariance matrix, given both classes and s
def covarianceMatrix(class1,s1,class2,s2):
    return ((len(class1)/len(class1 + class2))*s1) + ((len(class2)/len(class1 + class2))*s2)

#finding coeffecients for w and w0
def coefficientMatrices(phi,mean0,mean1,covariance):
    w = (covariance**-1)*(mean0 - mean1)
    w_0 = -0.5*(mean0.transpose()*(covariance**-1)*mean0) + 0.5*(mean1.transpose()*(covariance**-1)*mean1) + math.log(phi/(1-phi))   
    return[w,w_0]

#method given a data point, gives a value between 0 and 1 for the classification
def GDA_classification(data_point,w,w0):
    matrix = np.matrix(data_point).transpose()
    y = w.transpose()*matrix + w0
    return 1/(1 + math.exp(-y))




# initializing variables
phi=Phi(positive_class,negative_class)
mean0=meanMatrix(negative_class)
mean1=meanMatrix(positive_class)
s0=sValue(negative_class)
s1=sValue(positive_class)
covariance=covarianceMatrix(positive_class,s0,negative_class,s1)


#finding weights
w=coefficientMatrices(phi,mean1,mean0,covariance)[0]
w0=coefficientMatrices(phi,mean1,mean0,covariance)[1]


# method given dataset, and weights, gives all the classification evaluations, such as accuracy, precision, and f measure, wtc, wtc
def classifier_evaluation(dataset,w,w0):
    
    #initializing all measurement citeria
    correct = 0
    incorrect = 0
    true_pos = 0
    true_neg = 0
    false_pos = 0
    false_neg = 0
    for point in dataset:
        if GDA_classification(point[1],w,w0) > 0.5 :
        
            if point[0] == 'Positive':
                correct += 1
                true_pos += 1
            else:
                incorrect += 1
                false_pos += 1
        else:

            if point[0] == 'Negative':
                correct += 1
                true_neg += 1
            else:
                incorrect += 1
                false_neg += 1
    

    accuracy = correct / (correct + incorrect)
    precision = true_pos/(true_pos + false_pos)
    recall = true_pos/(true_pos + false_neg)
    F_measure = (2*precision*recall) / (precision + recall)

    # Reporting the coefficient matrix learned by the classifier and the results
    print("Accuracy: = ", accuracy)
    print("Precision: = ", precision)
    print("Recall: ", recall)
    print("F-measure: ", F_measure)
    print("Weights learned: W=", w)
    print("W0=", w0)


#calling method on training set
classifier_evaluation(test_set,w,w0)


Accuracy: =  0.96
Precision: =  0.9670050761421319
Recall:  0.9525
F-measure:  0.9596977329974811
Weights learned: W= [[ 14.1902841 ]
 [ -8.29689286]
 [ -5.01320057]
 [ -2.63225818]
 [ -9.24777763]
 [ -4.32177611]
 [ 15.67718196]
 [-23.88023661]
 [-28.18375148]
 [  9.21204196]
 [-12.8650161 ]
 [-11.47585923]
 [ 14.95966034]
 [ 12.30936621]
 [ -5.82786657]
 [ 12.99760802]
 [ 28.04494654]
 [ -6.78545758]
 [  0.13021411]
 [ -5.14591346]]
W0= [[26.6463063]]


# Question 3: K Nearest Neighbors

In [ ]:

# method for classification, given both classes and k, 
def KNN_classification(tag,k,class1,class2):
    distances=[]
    data=class1+class2
    
    for data_points in data:
        distances.append((abs(euclidean(tag, data_points[1])), data_points[0]))
        
    distances.sort()
    
    class1_count=0
    class2_count=0
    
    for i in range(k):
        if distances[i][1]==class1[0][0]:
            class1_count +=1
        else:
            class2_count +=1
            
    if class1_count == class2_count:

            if random.random() <= 0.5:

                return class1[0][0]

            else:

                return class2[0][0]

    elif class1_count > class2_count:

            return class1[0][0]

    else:

            return class2[0][0]


# method for evaluation for each KNN given amount of k
def KNN_Evaluation(amount):       
    ks = [x for x in range(1, amount)]
    accuracy_list = []
    precision_list = []
    recall_list = []
    F_measure_list = []

    # for loop to go through all the ks
    for k in ks:
        
        #initializse evaluation criterion
        correct= 0
        incorrect = 0
        true_pos = 0
        true_neg = 0
        false_pos = 0
        false_neg = 0

        for data_point in test_set:
            predicted_class = KNN_classification(data_point[1], k,positive_class,negative_class)
            if predicted_class == "Positive":
                if data_point[0] == "Positive":
                    correct += 1
                    true_pos += 1
                else:
                    incorrect += 1
                    false_pos += 1
            else:
                if data_point[0] == "Negative":
                    correct += 1
                    true_neg += 1
                else:
                    incorrect += 1
                    false_neg += 1

        accuracy_list.append(correct / (correct + incorrect))
        precision = true_pos/(true_pos + false_pos)
        precision_list.append(precision)
        recall = true_pos/(true_pos + false_neg)
        recall_list.append(recall)
        F_measure_list.append((2*precision*recall) / (precision + recall))


KNN_Evaluation(20) 
    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


# Question 4: Generate Data Part  2

In [79]:
# import all datasets needed
DS2_c1_m1=getData('DS2_c1_m1.txt')
DS2_c1_m2=getData('DS2_c1_m2.txt')
DS2_c1_m3=getData('DS2_c1_m3.txt')
DS2_c2_m1=getData('DS2_c2_m1.txt')
DS2_c2_m2=getData('DS2_c2_m2.txt')
DS2_c2_m3=getData('DS2_c2_m3.txt')
DS2_Cov1=getData('DS2_Cov1.txt')
DS2_Cov2=getData('DS2_Cov2.txt')
DS2_Cov3=getData('DS2_Cov3.txt')


#using same function from q2 but for new dataset
def splitData2(test,validation,train,class1,class2):
    np.random.shuffle(class1)
    np.random.shuffle(class2)

    test_set = [i for i in class1[0:int((len(class1)*test))]] + [j for j in class2[0:int((len(class2)*test))]]
    np.random.shuffle(test_set)
    pd.DataFrame(test_set).to_csv('DS2_test') 
    validation_set = [x for x in class1[int((len(class1)*test)):int(((len(class1)*test)+(len(class1)*validation)))]] + [y for y in class2[int((len(class2)*test)):int(((len(class2)*test)+(len(class2)*validation)))]]
    np.random.shuffle(validation_set)
    pd.DataFrame(validation_set).to_csv('DS2_valid')  
    class1 = class1[int(((len(class1)*test)+(len(class1)*validation))):len(class1)]    
    class2 = class2[int(((len(class2)*test)+(len(class2)*validation))):len(class2)]   
    train_set = class1 + class2
    np.random.shuffle(train_set)
    
    pd.DataFrame(train_set).to_csv('DS2_train')
    
    return[test_set,validation_set,train_set]

DS2_negative_class=[]
DS2_positive_class=[]

for i in range(2001):
    prob= random.random()
    
    if prob <= 0.1:
        DS2_negative_class.append(('Negative',np.random.multivariate_normal(DS2_c2_m1,DS2_Cov1)))
        DS2_positive_class.append(('Positive',np.random.multivariate_normal(DS2_c1_m1,DS2_Cov1)))
    elif prob <= 0.52:
        DS2_negative_class.append(('Negative',np.random.multivariate_normal(DS2_c2_m2,DS2_Cov2)))
        DS2_positive_class.append(('Positive',np.random.multivariate_normal(DS2_c1_m2,DS2_Cov2)))
    else:
        DS2_negative_class.append(('Negative',np.random.multivariate_normal(DS2_c2_m3,DS2_Cov3)))
        DS2_positive_class.append(('Positive',np.random.multivariate_normal(DS2_c1_m3,DS2_Cov3)))
    
    
np.random.shuffle(DS2_negative_class)
np.random.shuffle(DS2_positive_class)
 
DS2=DS2_negative_class+DS2_positive_class
np.random.shuffle(DS2)
pd.DataFrame(DS2).to_csv('DS2')


#intitializing test, validation, and trainingset
DS2_test_set=splitData2(.2,.2,.6,DS2_negative_class,DS2_positive_class)[0]
DS2_validation_set=splitData2(.2,.2,.6,DS2_negative_class,DS2_positive_class)[1]
DS2_train_set=splitData2(.2,.2,.6,DS2_negative_class,DS2_positive_class)[2]



# Question 5: Q2 & Q3 with DS2

In [80]:
DS2_phi=Phi(DS2_positive_class,DS2_negative_class)
DS2_mean0=meanMatrix(DS2_negative_class)
DS2_mean1=meanMatrix(DS2_positive_class)
DS2_s0=sValue(DS2_negative_class)
DS2_s1=sValue(DS2_positive_class)
DS2_covariance=covarianceMatrix(DS2_positive_class,DS2_s0,DS2_negative_class,DS2_s1)


#finding weights
D2_w=coefficientMatrices(DS2_phi,DS2_mean1,DS2_mean0,DS2_covariance)[0]
D2_w0=coefficientMatrices(DS2_phi,DS2_mean1,DS2_mean0,DS2_covariance)[1]


classifier_evaluation(DS2_test_set,D2_w,D2_w0)



Accuracy: =  0.54625
Precision: =  0.5461346633416458
Recall:  0.5475
F-measure:  0.5468164794007491
Weights learned: W= [[-0.05292645]
 [ 0.04098204]
 [-0.02845562]
 [ 0.01662299]
 [-0.05905746]
 [-0.01920297]
 [-0.02989649]
 [-0.03463659]
 [-0.00241258]
 [ 0.03461814]
 [-0.00062787]
 [-0.07470424]
 [-0.0405562 ]
 [ 0.03792138]
 [ 0.04563976]
 [ 0.00071448]
 [ 0.04253994]
 [ 0.02844131]
 [ 0.0410429 ]
 [ 0.00287101]]
W0= [[0.05923283]]


# Question 6: DS1 vs DS2

### DS1 Dataset seems to contrain less variantion among the data leading to better classifications for GDA, while DS2 seems to be doing less better, this variation is bad for a linear classifier but better for KNN because it accounts of variation within the dataset, this KNN is better for DS2 than DS1